In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [4]:
driver = webdriver.Chrome('E:\\BigData\\crawling\\chromedriver.exe')

In [5]:
url = 'https://www.starbucks.co.kr/store/store_map.do'

In [7]:
driver.get(url)

In [16]:
driver.find_element_by_xpath('//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a').click()
driver.find_element_by_xpath('//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a').click()

In [18]:
driver.find_element_by_xpath('//*[@id="mCSB_2_container"]/ul/li[1]/a').click()

### 페이지 html 추출
    1. html = driver.page_source => 해당page html 추출
    2. bs활용 parser

In [22]:
html = driver.page_source
soup = bs(html,'html.parser')
soup

<html lang="ko"><head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="website" property="og:type"/>
<meta content="Starbucks" property="og:title"/>
<meta content="https://www.starbucks.co.kr/" property="og:url"/>
<meta content="https://image.istarbucks.co.kr/common/img/kakaotalk.png" property="og:image"/>
<meta content="Starbucks" property="og:description"/>
<title id="titleJoin">Starbucks Coffee Korea</title>
<link href="https://image.istarbucks.co.kr/common/img/common/favicon.ico?v=200828" rel="shortcut icon" type="image/ico"/> <!-- 20200827 파비콘 교체 및 CDN 변수처리 -->
<link href="/common/css/reset.css" rel="stylesheet"/>
<link href="/common/css/style.css?v=200130" rel="stylesheet"/>
<link href="/common/css/jquery.bxslider.css" rel="stylesheet"/>
<link href="/common/css/idangerous.swiper.css" rel="stylesheet"/>
<link 

##### 서울지역 모든 스벅 매장정보 추출

In [56]:
s_l = soup.select('ul.quickSearchResultBoxSidoGugun > li')
len(s_l)

528

In [57]:
s_l[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong>역삼아레나빌딩  <img alt="" class="setStoreFavBtn mCS_img_loaded" data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [146]:
# 매장이름
s_l[0].select('strong')[0].text.strip()  

'역삼아레나빌딩'

In [147]:
# 위도
s_l[0]['data-lat'].strip()

'37.501087'

In [148]:
# 경도
s_l[0]['data-long'].strip()

'127.043069'

In [145]:
# 매장 유형
s_l[0].select('i')[0]['class'][0].split('_')[1]

'general'

In [149]:
s_l[11].select('i')[0]['class'][0]

'pin_reserve'

In [150]:
#매장 주소
str(s_l[0].select('p')[0]).split('<br/>')[0].split('>')[1]

'서울특별시 강남구 언주로 425 (역삼동)'

In [151]:
# 매장 번호
str(s_l[0].select('p')[0]).split('<br/>')[1].split('<')[0]

'1522-3232'

##### 모든매장의 이름, 위도,경도, 유형, 주소, 번호 추출 및 리스트 저장

In [139]:
Name = []
Lat = []
Long = []
Type = []
Address = []
Number = []
for i in s_l:
    Name.append(i.select('strong')[0].text.strip())
    Lat.append(i['data-lat'])
    Long.append(i['data-long'])
    Type.append(i.select('i')[0]['class'][0].split('_')[1])
    Address.append(str(i.select('p')[0]).split('<br/>')[0].split('>')[1])
    Number.append(str(i.select('p')[0]).split('<br/>')[1].split('<')[0])
D = {'Name':Name, 'Lat':Lat,'Long':Long,'Type':Type,'Address':Address,'Number':Number}

In [156]:
df = pd.DataFrame(D)
df.head()

,Name,Lat,Long,Type,Address,Number
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [217]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Name     528 non-null    object 
 1   Lat      528 non-null    float64
 2   Long     528 non-null    float64
 3   Type     528 non-null    object 
 4   Address  528 non-null    object 
 5   Number   528 non-null    object 
dtypes: float64(2), object(4)
memory usage: 24.9+ KB


In [216]:
df['Lat'] = df.Lat.astype(float)
df['Long'] =df.Long.astype(float)

In [170]:
df.to_csv('./data/starbucks_list.csv')

##### 매장위치 표시

In [142]:
import folium

In [219]:
lat = df.Lat.mean()   ##  mean에서 error
long = df.Long.mean()

m = folium.Map(location = [lat, long], zoom_start=11)

for i in range(528):
    lat = df.Lat[i]
    long = df.Long[i]
    name = df.Name[i]
    folium.Marker([lat, long],tooltip = name).add_to(m)
m

##### 구별 시군구목록 데이터, 구별 주민등록인구데이터, 구별 사업체현황데이터 활용 스타벅스 매장 수와의 관계 도출
    1. 자료수집 : 서울시 열린데이터광장에서 open api로 수집

In [193]:
url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25'.format('4a517950776c65653535616d586264')

In [194]:
url

'http://openapi.seoul.go.kr:8088/4a517950776c65653535616d586264/json/SdeTlSccoSigW/1/25'

In [300]:
import requests
import json   # json형식 파일 저장식 필요

In [196]:
res = requests.get(url)

In [221]:
df_s = pd.DataFrame(res.json()['SdeTlSccoSigW']['row'])
df_s.head()

,OBJECTID,SIG_CD,SIG_KOR_NM,SIG_ENG_NM,ESRI_PK,LAT,LNG
0,1.0,11320,도봉구,Dobong-gu,0.0,37.6658609,127.0317674
1,2.0,11380,은평구,Eunpyeong-gu,1.0,37.6176125,126.9227004
2,3.0,11230,동대문구,Dongdaemun-gu,2.0,37.5838012,127.0507003
3,4.0,11590,동작구,Dongjak-gu,3.0,37.4965037,126.9443073
4,5.0,11545,금천구,Geumcheon-gu,4.0,37.4600969,126.9001546


In [224]:
k = '4a517950776c65653535616d586264'

# api로 서비스데이터 요청 및 리턴하는 함수
def soad(u,s):   
    data_list = None
    try :
        result_dict = requests.get(u).json()
        result_data = result_dict[s]
        code = result_data['RESULT']['CODE']
        if code == 'INFO-000':
            data_list = result_data['row']
    except:pass
    return data_list

In [226]:
k = '4a517950776c65653535616d586264'
url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25'.format(k)
s = 'SdeTlSccoSigW'

data = soad(url,s)
data

[{'OBJECTID': 1.0,
  'SIG_CD': '11320',
  'SIG_KOR_NM': '도봉구',
  'SIG_ENG_NM': 'Dobong-gu',
  'ESRI_PK': 0.0,
  'LAT': '37.6658609',
  'LNG': '127.0317674'},
 {'OBJECTID': 2.0,
  'SIG_CD': '11380',
  'SIG_KOR_NM': '은평구',
  'SIG_ENG_NM': 'Eunpyeong-gu',
  'ESRI_PK': 1.0,
  'LAT': '37.6176125',
  'LNG': '126.9227004'},
 {'OBJECTID': 3.0,
  'SIG_CD': '11230',
  'SIG_KOR_NM': '동대문구',
  'SIG_ENG_NM': 'Dongdaemun-gu',
  'ESRI_PK': 2.0,
  'LAT': '37.5838012',
  'LNG': '127.0507003'},
 {'OBJECTID': 4.0,
  'SIG_CD': '11590',
  'SIG_KOR_NM': '동작구',
  'SIG_ENG_NM': 'Dongjak-gu',
  'ESRI_PK': 3.0,
  'LAT': '37.4965037',
  'LNG': '126.9443073'},
 {'OBJECTID': 5.0,
  'SIG_CD': '11545',
  'SIG_KOR_NM': '금천구',
  'SIG_ENG_NM': 'Geumcheon-gu',
  'ESRI_PK': 4.0,
  'LAT': '37.4600969',
  'LNG': '126.9001546'},
 {'OBJECTID': 6.0,
  'SIG_CD': '11530',
  'SIG_KOR_NM': '구로구',
  'SIG_ENG_NM': 'Guro-gu',
  'ESRI_PK': 5.0,
  'LAT': '37.4954856',
  'LNG': '126.858121'},
 {'OBJECTID': 7.0,
  'SIG_CD': '11110',
  '

In [309]:
df_1 = pd.DataFrame(data).iloc[ : ,[1,2,-2,-1]]
df_1

,JACHIGU,DONG,JONGSAJASU_8,SAEOPCHESU_10
0,합계,합계,279921,126785
1,종로구,소계,7841,6635
2,종로구,사직동,2281,856
3,종로구,삼청동,84,261
4,종로구,부암동,37,125
...,...,...,...,...
445,강동구,둔촌1동,123,15
446,강동구,둔촌2동,611,236
447,강동구,암사1동,636,279
448,강동구,천호2동,505,683


In [310]:
df_1.columns=['시군구코드', '시군구명', '위도', '경도']
df_1

,시군구코드,시군구명,위도,경도
0,합계,합계,279921,126785
1,종로구,소계,7841,6635
2,종로구,사직동,2281,856
3,종로구,삼청동,84,261
4,종로구,부암동,37,125
...,...,...,...,...
445,강동구,둔촌1동,123,15
446,강동구,둔촌2동,611,236
447,강동구,암사1동,636,279
448,강동구,천호2동,505,683


In [ ]:
df_s.to_excel('E:\\BigData\\eda\\data\\/4_2_seoul_list.xlsx',index = False)

In [311]:
df_1.head(1)

,시군구코드,시군구명,위도,경도
0,합계,합계,279921,126785


In [257]:
url = 'http://openapi.seoul.go.kr:8088/{}/json/octastatapi419/1/26'.format(k)
s = 'octastatapi419'

pop_d_l = soad(url,s)

In [263]:
df_p = pd.DataFrame(pop_d_l)
df_p.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,﻿2010.1-4,합계,"4,121,193","10,464,171","5,181,117","5,283,054","10,208,670","5,059,504","5,149,166","255,501","121,613","133,888",2.48,"956,414"
1,2010.1-4,종로구,"74,300","177,023","88,225","88,798","168,158","84,088","84,070","8,865","4,137","4,728",2.26,"20,925"
2,2010.1-4,중구,"58,472","138,102","69,175","68,927","129,786","65,216","64,570","8,316","3,959","4,357",2.22,"16,208"
3,2010.1-4,용산구,"107,407","250,844","123,295","127,549","238,366","116,764","121,602","12,478","6,531","5,947",2.22,"28,436"
4,2010.1-4,성동구,"126,942","316,113","158,228","157,885","308,144","154,389","153,755","7,969","3,839","4,130",2.43,"30,704"


In [265]:
df_ps = df_p[df_p.JACHIGU != '합계']

In [312]:
df_2 = df_ps.iloc[:,[1,3]]
df_2.head(1)

,JACHIGU,GYE_1
1,종로구,"177,023"


In [313]:
df_2.columns = ['시군구명','주민등록인구']
df_2.head(1)

,시군구명,주민등록인구
1,종로구,"177,023"


In [284]:
df_pf.to_excel('E:\\BigData\\eda\\data\\4_3_sgg_pop.xlsx', index = False)

##### 서울시 동별 사업체현황 요청

In [280]:
url = 'http://openapi.seoul.go.kr:8088/{}/json/octastatapi104/1/450'.format(k)
s = 'octastatapi104'

data = soad(url,s)
data

[{'GIGAN': '2015',
  'JACHIGU': '합계',
  'DONG': '합계',
  'SAEOPCHESU_1': '820658',
  'YEOSEONGDAEPYOJA': '272567',
  'GYE': '5108828',
  'NAM': '2887719',
  'YEO': '2221109',
  'SAEOPCHESU_2': '17',
  'JONGSAJASU_1': '491',
  'SAEOPCHESU_3': '23',
  'JONGSAJASU_2': '69',
  'SAEOPCHESU_4': '62211',
  'JONGSAJASU_3': '287414',
  'SAEOPCHESU_5': '137',
  'JONGSAJASU_4': '8713',
  'SAEOPCHESU_6': '463',
  'JONGSAJASU_5': '6712',
  'SAEOPCHESU_7': '22695',
  'JONGSAJASU_6': '360192',
  'SAEOPCHESU_8': '234494',
  'JONGSAJASU_7': '912406',
  'SAEOPCHESU_9': '93642',
  'JONGSAJASU_8': '279921',
  'SAEOPCHESU_10': '126785'},
 {'GIGAN': '2015',
  'JACHIGU': '종로구',
  'DONG': '소계',
  'SAEOPCHESU_1': '40871',
  'YEOSEONGDAEPYOJA': '13472',
  'GYE': '269106',
  'NAM': '152766',
  'YEO': '116340',
  'SAEOPCHESU_2': '1',
  'JONGSAJASU_1': '16',
  'SAEOPCHESU_3': '2',
  'JONGSAJASU_2': '7',
  'SAEOPCHESU_4': '4281',
  'JONGSAJASU_3': '11734',
  'SAEOPCHESU_5': '6',
  'JONGSAJASU_4': '387',
  'SAEOPCHES

In [407]:
df_3a = pd.DataFrame(data)
df_3a

,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,SAEOPCHESU_2,JONGSAJASU_1,...,JONGSAJASU_4,SAEOPCHESU_6,JONGSAJASU_5,SAEOPCHESU_7,JONGSAJASU_6,SAEOPCHESU_8,JONGSAJASU_7,SAEOPCHESU_9,JONGSAJASU_8,SAEOPCHESU_10
0,2015,합계,합계,820658,272567,5108828,2887719,2221109,17,491,...,8713,463,6712,22695,360192,234494,912406,93642,279921,126785
1,2015,종로구,소계,40871,13472,269106,152766,116340,1,16,...,387,9,179,358,25269,17447,46970,1418,7841,6635
2,2015,종로구,사직동,3551,1092,51425,29781,21644,1,16,...,2,2,32,38,4158,563,4074,280,2281,856
3,2015,종로구,삼청동,738,354,4736,2374,2362,-,-,...,-,-,-,6,15,273,864,20,84,261
4,2015,종로구,부암동,540,224,3635,1714,1921,-,-,...,-,-,-,22,53,104,523,33,37,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,2015,강동구,둔촌1동,416,176,1361,785,576,-,-,...,91,-,-,11,14,112,208,108,123,15
446,2015,강동구,둔촌2동,1565,548,9063,4685,4378,-,-,...,-,-,-,70,531,342,2487,257,611,236
447,2015,강동구,암사1동,1892,695,5313,2869,2444,-,-,...,-,-,-,73,605,419,1107,421,636,279
448,2015,강동구,천호2동,3685,1452,14534,6972,7562,-,-,...,-,3,10,115,418,1296,4518,358,505,683


In [408]:
df_3 = df_3a[df_3a.DONG == '소계'].iloc[:,[1,5,3]]

In [409]:
df_3.columns=['시군구명','종사자수','사업체수']

In [410]:
df_3 = df_3.reset_index(drop = True)

In [411]:
df_3.to_excel('E:\\BigData\\eda\\data\\4_3_sgg_biz.xlsx', index = False)

In [412]:
df_3.head(1)

,시군구명,종사자수,사업체수
0,종로구,269106,40871


##### 데이터수집 & 기초처리 & 저장 완료

In [334]:
df = pd.read_excel('E:\\BigData\\eda\\data\\4_1_seoul_starbucks_list.xlsx')
df.head(2)

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232


##### '구'정보를 담는 '시군구명'칼럼 생성

In [339]:
df['시군구명'] =  df.주소.str.split(' ',expand = True)[1]

In [341]:
df.to_excel('E:\\BigData\\eda\\data\\4_4_seoul_starbucks_list.xlsx',index=False)

In [414]:
df1 = pd.read_excel('E:\\BigData\\eda\\data\\4_2_seoul_sgg_list.xlsx') # 시군구코드, 시군구명, 위도, 경도

##### df1에 '스타벅스매장수' col을 추가

In [415]:
d = dict(df.시군구명.value_counts())
d

{'강남구': 81,
 '중구': 54,
 '서초구': 47,
 '종로구': 37,
 '영등포구': 34,
 '마포구': 32,
 '송파구': 32,
 '서대문구': 19,
 '용산구': 19,
 '강서구': 16,
 '광진구': 16,
 '강동구': 15,
 '양천구': 15,
 '성북구': 13,
 '노원구': 12,
 '성동구': 12,
 '동작구': 11,
 '구로구': 11,
 '관악구': 11,
 '금천구': 11,
 '동대문구': 8,
 '은평구': 8,
 '중랑구': 7,
 '강북구': 5,
 '도봉구': 2}

In [416]:
df1['스타벅스매장수'] = df1.시군구명.apply(lambda x : d[x])

In [417]:
df1 # 시군구코드, 시군구명, 위도, 경도, 스타벅스매장수

,시군구코드,시군구명,위도,경도,스타벅스매장수
0,11320,도봉구,37.665861,127.031767,2
1,11380,은평구,37.617612,126.922700,8
2,11230,동대문구,37.583801,127.050700,8
3,11590,동작구,37.496504,126.944307,11
4,11545,금천구,37.460097,126.900155,11
5,11530,구로구,37.495486,126.858121,11
6,11110,종로구,37.599100,126.986149,37
7,11305,강북구,37.646995,127.014716,5
8,11260,중랑구,37.595379,127.093967,7
9,11680,강남구,37.495985,127.066409,81


##### df1에 구별인구데이터 병합

In [418]:
df_2

,시군구명,주민등록인구
1,종로구,"177,023"
2,중구,"138,102"
3,용산구,"250,844"
4,성동구,"316,113"
5,광진구,"386,292"
6,동대문구,"374,039"
7,중랑구,"431,283"
8,성북구,"484,359"
9,강북구,"342,669"
10,도봉구,"371,095"


In [419]:
df1

,시군구코드,시군구명,위도,경도,스타벅스매장수
0,11320,도봉구,37.665861,127.031767,2
1,11380,은평구,37.617612,126.922700,8
2,11230,동대문구,37.583801,127.050700,8
3,11590,동작구,37.496504,126.944307,11
4,11545,금천구,37.460097,126.900155,11
5,11530,구로구,37.495486,126.858121,11
6,11110,종로구,37.599100,126.986149,37
7,11305,강북구,37.646995,127.014716,5
8,11260,중랑구,37.595379,127.093967,7
9,11680,강남구,37.495985,127.066409,81


In [420]:
df1 = pd.merge(df1, df_2, on='시군구명')

In [421]:
df1.head(1)

,시군구코드,시군구명,위도,경도,스타벅스매장수,주민등록인구
0,11320,도봉구,37.665861,127.031767,2,"371,095"


##### 종사자수, 사업체수 col 추가

In [422]:
df1 = pd.merge(df1,df_3, on = '시군구명')
df1.head(1)

,시군구코드,시군구명,위도,경도,스타벅스매장수,주민등록인구,종사자수,사업체수
0,11320,도봉구,37.665861,127.031767,2,"371,095",68669,18455
